In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl


##  Clarification About the Misunderstanding

Short version: In the first version, **I misunderstood what the “heat” variable actually meant**.  
I initially thought “heat” represented **heating power** (how strong the heater runs).  

So in practice:
- Earlier version → “how many watts should we heat with?”
- Current version → “what temperature band should the room move toward?”

Because of this semantic mismatch, the first results looked strange.  
To fix it, I had to either remove or rewrite several rules from scratch.  

I tried to explainn everything bellow here:

##  Merging Rules That Actually Meant the Same Thing

In the initial rule set, we had repeated patterns (different wording, same behavior):

- “user feels cold → increase”
- “temperature is cold → increase”
- “user slightly cold + temp is low → increase”

When thinking in terms of **heating power**, it made sense to write these separately.  
But when switching to a **target temperature** interpretation, all of them reduce to:

- “In this situation, choose a warmer target temperature category.”

So I merged several of these similar rules into a single, more general rule.

The same thing happened for **hot scenarios**:

- user feels too hot  
- temperature already too hot  
- temperature is hot + humidity high  

All of these essentially said:  
- “Lower the setpoint.”

So these also became fewer and cleaner rules.


##  Simplifying Eco-Related Rules

Old logic:
- eco acted like “a brake on heating power”  
- meaning: many small rules like “if eco is high → reduce power a bit.”

New logic:
- eco means “you prefer a cooler target temperature”
- so the setpoint is simply picked one category lower.

This allowed us to:
- remove 2–3 “eco + power” combination rules
- express the same behavior with just a couple of clear rules

This simplification also reduced the total rule count.


## Summary: What I removed vs. what I added

**What I removed:**
- repetitive rules that produced the same outcome  
- power-based rules that didn’t make sense once we treat “heat” as target temperature  
- unnecessary fine-grained variations that the MF design already covered

**What I added:**
- fewer rules but clearer logic  
- behavior expressed directly in terms of temperature categories  
- a rule set that actually matches the meaning of our membership functions  
- a system that’s easier to interpret and maintain

Because of all this cleanup and restructuring:  
**the rule count went from 17 down to 14.**


In [ ]:
# FUZZY RULE SET 
# "target" means the tempature that system desired.

# A) User feels too cold → raise the target temperature
rule1  = ctrl.Rule(feeling['too cold'] & ecology['not eco'],heat['rather hot'])      
# user feels very cold and ecology is not a concern so choose a warmer target

rule2  = ctrl.Rule(feeling['too cold'] & ecology['medium'],heat['comfortable'])      
# user feels cold but ecology matters somewhat so increase target moderately

rule3  = ctrl.Rule(feeling['too cold'] & ecology['eco'],heat['comfortable'])      
# even with eco preference, keep the target at a comfortable level

# If both the room and the user are cold as a result increase target further
rule4  = ctrl.Rule(temperature['too cold'] & feeling['too cold'],heat['rather hot'])      
# cold room + cold feeling so set a warmer target


# B) User feels okay so stay near comfortable, eco slightly lowers target
rule5  = ctrl.Rule(feeling['okay'] & temperature['comfortable'] & ecology['not eco'],heat['comfortable'])      
# everything is normal so keep the target comfortable

rule6  = ctrl.Rule(feeling['okay'] & temperature['comfortable'] & ecology['medium'],heat['chilly'])      
# medium eco preference so choose a slightly cooler target

rule7  = ctrl.Rule(feeling['okay'] & temperature['comfortable'] & ecology['eco'],heat['chilly'])      
# eco preference so prefer a cooler target


# If the room is a bit cold but the user feels okay as a result raise target slightly
rule8  = ctrl.Rule(feeling['okay'] & temperature['chilly'],heat['comfortable'])      
# chilly room but user is okay so target should be slightly warmer


# If the room is warm while the user feels okay so lower target a bit
rule9  = ctrl.Rule(feeling['okay'] & temperature['rather hot'],heat['chilly'])      
# user is okay but room is somewhat hot so choose a cooler target


# If the room is too hot as a result significantly lower the target
rule10 = ctrl.Rule(feeling['okay'] & temperature['too hot'],heat['too cold'])      
# room is too hot so choose the coldest target


# C) Humidity effect: high humidity makes cold feel colder
rule11 = ctrl.Rule(temperature['chilly'] & humidity['high'] & feeling['okay'],heat['comfortable'])     
# humidity exaggerates cold so raise target slightly


# D) User feels too hot so lower the target temperature
rule12 = ctrl.Rule(feeling['too hot'],heat['too cold'])      
# user is too hot so choose the coldest target


# E) Sensor-based safety rules
rule13 = ctrl.Rule(temperature['too hot'],heat['too cold'])      
# extremely hot room so force target down

rule14 = ctrl.Rule(ecology['eco'] & temperature['rather hot'],heat['too cold'])     
# eco preference + warm room so push target down further


# control system
heating_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4,
    rule5, rule6, rule7, rule8, rule9, rule10,
    rule11, rule12, rule13, rule14
])

heating_sim = ctrl.ControlSystemSimulation(heating_ctrl)


In [ ]:
# ===== TEST SIMULATION EXAMPLE =====

# Example scenario:
# - living room, membership already defined above
# - temperature: 18°C (a bit cold)
# - humidity: 60% (slightly humid)
# - feeling: 3/10 (closer to 'too cold' region, but overlapping with 'okay')
# - ecology: 2/10 (eco-friendly)

heating_sim.input['temperature'] = 18
heating_sim.input['humidity'] = 60
heating_sim.input['feeling'] = 3
heating_sim.input['ecology'] = 2

heating_sim.compute()
print("Computed heat output:", heating_sim.output['heat'])